In [4]:
import pandas as pd

In [5]:
!pip install sentence-transformers

In [6]:
df = pd.read_csv('./train_data_en.csv')

In [7]:
df.head()

,Unnamed: 0,Задача en,Обстановка en,Оптимальный план en,Предсказанный план,Успех предсказанного плана
0,3839,Prepare salted popcorn,"The kitchen has a microwave, a bag of popcorn,...",Open the door. Enter the kitchen. Close the do...,Walk to the microwave. Open the microwave. G...,1
1,4726,Boil the potatoes.,"kitchen, potatoes, drawer, pan, sink with tap,...",Find a table. Find a saucepan on the table. Gr...,Find a box. Open the box. Find potatoes. Gra...,0
2,1921,Put the baby to bed,"children's room, bed, blanket, pillow, favorit...",Open the door. Enter the children's room. Clos...,Open the door. Enter the children's room. Cl...,0
3,2067,Collect quail eggs from the farm barn.,"Farm, barn, quails, quail nests, quail eggs, t...",Find a barn on a farm. Walk to the barn. Find ...,Walk to the barn. Open the door. Enter the b...,1
4,1813,Clean the refrigerator,In the kitchen there is a refrigerator with fo...,Walk to the refrigerator. Unplug the refrigera...,Walk to the refrigerator. Unplug the refrige...,0


In [8]:
import os
import re
import zipfile
import importlib

import gdown
import numpy as np
import torch
from natsort import natsorted
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import gc
import torch
import torch
import torch.nn.functional as F
from torchvision import transforms
from tqdm.auto import trange
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from  torch import nn
import math
import itertools
import torchvision
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn.init as init
from torch import nn
from torch import distributions
from torch.distributions import MultivariateNormal, Uniform, TransformedDistribution, SigmoidTransform
from torch.nn.parameter import Parameter

In [9]:
from sentence_transformers import SentenceTransformer, util
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1",  device=device)

model = model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
def transforms(df, train=True):
    text_fields = ['Задача en', 'Обстановка en', 'Оптимальный план en', 'Предсказанный план']
    emb = [model.encode(df[i]) for i in text_fields]
    new_feature_1 = np.array([util.dot_score(emb[0][i], emb[-1][i]) for i in range(df.shape[0])])
    new_feature_1 = np.squeeze(new_feature_1)
    X = np.hstack(emb)
    X = np.column_stack((X, new_feature_1))
    y = df['Успех предсказанного плана'] if train else None
    return X, y

In [11]:
X, y = transforms(df, True)

In [12]:
X.shape, y.shape

((3383, 1537), (3383,))

In [13]:
test_df = pd.read_csv('./test_data_en.csv')

X_test, y_test = transforms(df, False)

In [14]:
X_test.shape, y_test.shape

((3383, 1537), (3383,))

In [16]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 9.8 MB/s eta 0:00:00


In [17]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

catboost_params = {
    'task_type': 'GPU',
    'loss_function': 'Logloss',
    'random_seed': 42,
    'od_type': 'Iter',
    'early_stopping_rounds': 300,
    'random_strength': 0.5,
    'verbose': 100,
    'eval_metric': 'Accuracy',
}

cb = CatBoostClassifier(**catboost_params)

cb.fit(X, y, eval_set=(X_test, y_test), use_best_model=True)

Learning rate set to 0.0642
0:	learn: 0.6854863	test: 0.6854863	best: 0.6854863 (0)	total: 367ms	remaining: 6m 6s
100:	learn: 0.7853976	test: 0.7853976	best: 0.7853976 (100)	total: 14.9s	remaining: 2m 12s
200:	learn: 0.8962459	test: 0.8962459	best: 0.8962459 (200)	total: 24.8s	remaining: 1m 38s
300:	learn: 0.9491576	test: 0.9491576	best: 0.9491576 (300)	total: 33.9s	remaining: 1m 18s
400:	learn: 0.9769435	test: 0.9769435	best: 0.9769435 (400)	total: 43.3s	remaining: 1m 4s
500:	learn: 0.9926101	test: 0.9926101	best: 0.9926101 (499)	total: 52.3s	remaining: 52.1s
600:	learn: 0.9979308	test: 0.9979308	best: 0.9979308 (569)	total: 1m 5s	remaining: 43.2s
700:	learn: 0.9991132	test: 0.9991132	best: 0.9991132 (675)	total: 1m 14s	remaining: 31.7s
800:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (793)	total: 1m 23s	remaining: 20.8s
900:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (793)	total: 1m 32s	remaining: 10.2s
999:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (793)	total: 1m 

In [18]:
from sklearn.metrics import accuracy_score

predictions = cb.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [20]:
cb.save_model('model.cb')

In [26]:
import argparse
import pandas as pd

import os
import re
import zipfile
import importlib

import gdown
import numpy as np
import torch
from natsort import natsorted
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import gc
import torch
import torch
import torch.nn.functional as F
from torchvision import transforms
from tqdm.auto import trange
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from  torch import nn
import math
import itertools
import torchvision
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn.init as init
from torch import nn
from torch import distributions
from torch.distributions import MultivariateNormal, Uniform, TransformedDistribution, SigmoidTransform
from torch.nn.parameter import Parameter
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score


def transforms(df, train=True):
    text_fields = ['Задача en', 'Обстановка en', 'Оптимальный план en', 'Предсказанный план']
    emb = [model.encode(df[i]) for i in text_fields]
    new_feature_1 = np.array([util.dot_score(emb[0][i], emb[-1][i]) for i in range(df.shape[0])])
    new_feature_1 = np.squeeze(new_feature_1)
    X = np.hstack(emb)
    X = np.column_stack((X, new_feature_1))
    y = df['Успех предсказанного плана'] if train else None
    return X, y

def predict(df):
    X_test, _ = transforms(df, False)
    model = CatBoostClassifier()
    model.load_model('model.cb')
    return model.predict(X_test)

def main(path_to_csv):
    df = pd.read_csv(path_to_csv)
    predictions = predict(df)
    predictions_df = pd.DataFrame(predictions, index=df.index)
    predictions_df.to_csv('labels_123.csv', index=False)

# if __name__ == "__main__":
#     parser = argparse.ArgumentParser()
#     parser.add_argument("path", type=str, help="Path to the CSV file")
#     args = parser.parse_args()

#     main(args.path)

In [27]:
main('test_data_en.csv')